In [1]:
import json
import pandas as pd
from ray_casting import point_intersects_multi_polygon

# Load data

In [2]:
with open('data/sf_zones.json', 'r') as f:
    zones = json.load(f)
zones_features = zones['features']

In [3]:
stations = pd.read_csv('data/sf_stations.csv')
stations['id'] = stations['id'].astype('int')

# Map stations and zones

In [4]:
class ZoneNotFoundException(Exception):
    pass

In [5]:
def find_zone(latitude, longitude):
    point = (longitude, latitude)
    for zone in zones_features:
        multi_polygon = zone['geometry']['coordinates']
        if point_intersects_multi_polygon(point, multi_polygon):
            return zone['properties']['id']
    raise ZoneNotFoundException

In [6]:
station2zone = {}
station_ids = stations.id
for station_id in station_ids:
    row = stations[stations.id == station_id].iloc[0]
    zone_id = find_zone(row.latitude, row.longitude)
    station2zone[station_id] = zone_id

In [7]:
def find_stations(zone):
    multi_polygon = zone['geometry']['coordinates']
    zone_stations = []
    for _, station in stations.iterrows():
        point = (station.longitude, station.latitude)
        if point_intersects_multi_polygon(point, multi_polygon):
            zone_stations.append(int(station.id))
    return zone_stations

In [8]:
zone2stations = {}
for zone in zones_features:
    zone_id = zone['properties']['id']
    zone_stations = find_stations(zone)
    zone2stations[zone_id] = zone_stations

In [9]:
with open('data/station2zone.json', 'w') as f:
    json.dump(station2zone, f)

In [10]:
with open('data/zone2stations.json', 'w') as f:
    json.dump(zone2stations, f)

Station 19 and zone with name "200 Sutter Street, Financial District, San Francisco" are supposed to overlap.

In [11]:
zone_overlap = list(filter(lambda feature: feature['properties']['name'] == '200 Sutter Street, Financial District, San Francisco', zones_features))[0]

In [12]:
station_overlap_id = 19
zone_overlap_id = zone_overlap['properties']['id']

In [13]:
print(zone_overlap_id)

1689


In [14]:
station_overlap = stations[stations.id == station_overlap_id].iloc[0]
print(find_zone(station_overlap.latitude, station_overlap.longitude))

1689


In [15]:
zone2stations[zone_overlap_id]

[19, 20]

Correct!